In [5]:
#skip
%run 00_basic.ipynb

In [1]:
%%writefile  ~/bin/pdfmerge.py
#!/usr/local/bin/python 
import os, PyPDF2, glob, datetime
from PyPDF2 import PdfFileReader, PdfFileMerger


def pdfmerge(dir="."):
    pdfWriter = PyPDF2.PdfFileWriter()

    for f in sorted(glob.glob(f"{dir}/*.pdf")):
        if ( f.find("out.pdf") >0):
            continue;
        inp = open(f, "rb")  # <== DO NOT USE "with" - you must keep it open 
        pdfReader = PyPDF2.PdfFileReader(inp)
        print(f"processing {f} ; #{pdfReader.numPages}")
        for pageNum in range(pdfReader.numPages):
            pageObj = pdfReader.getPage(pageNum)
            pdfWriter.addPage(pageObj)

    with open(f"{dir}/out.pdf", "wb") as out:
        pdfWriter.write(out)

def inJupyter():
    try: get_ipython; return 1; 
    except: return 0
#-----------------------------------------------------------------------------------
if __name__ == '__main__':
    if (not inJupyter()):
        t1 = datetime.datetime.now()
        pdfmerge()
        t2 = datetime.datetime.now()
    print(f"All Done in {str(t2-t1)} ***")

Overwriting /Users/e346104/bin/pdfmerge.py


In [16]:
#%%writefile  ~/bin/createbook.py
#!/usr/local/bin/python 
import os, PyPDF2, glob, datetime
from PyPDF2 import PdfFileReader, PdfFileMerger


ignore='''Utils 00_basic
'''.split()

CWD=os.path.abspath(os.getcwd())

def createbook(dir=".."):
    for f in sorted(glob.glob(f"{dir}/*.ipynb")):
        bn=os.path.splitext(os.path.basename(f))[0]
        if ( bn in ignore):
            print(f"ignoring {f}")
            continue;
        cmd = '''
        f"#Processing {f}"
        jupyter nbconvert --to html --execute {f} --output {CWD}/{bn}.html
        
        '''
        print()

def inJupyter():
    try: get_ipython; return 1; 
    except: return 0
#-----------------------------------------------------------------------------------
if __name__ == '__main__':
    if (not inJupyter()):
        t1 = datetime.datetime.now()
        createbook()
        t2 = datetime.datetime.now()
        print(f"All Done in {str(t2-t1)} ***")

In [56]:
file="300_NNTimeGenerators.ipynb"
with open(file, "rb") as f:
    d = f.read();
    
j=json.loads(d)
print(f'#cells : {len(j["cells"])}, nbformat: {j["nbformat"]}.{j["nbformat_minor"]}')
print(f'{d[0:100]}')
jc = j["cells"]
for i, js in enumerate(jc):
    jss=js['source']
    if ( len(jss) <= 0): continue
    if (js['source'][0].startswith("#skip")):
        print(f"{i}: Skipping {jss}")

#cells : 4, nbformat: 4.4
b'{\n "cells": [\n  {\n   "cell_type": "code",\n   "execution_count": 2,\n   "metadata": {},\n   "outputs": '
0: Skipping ['#skip\n', '%run 00_basic.ipynb']


In [116]:
from IPython.display import display, Markdown, Latex, HTML
js=jc[1]
jss=js['source']
if ( js['cell_type'] == 'markdown' ):
    print(f"Yes")
    out=Markdown("\n".join(jss))
    display(out)    

Yes


# Time Genreators in keras



In Time-series analysis, more often we need to transform the data into a different structure before it can be fed to LSTM.



The Keras DNN library provides the TimeseriesGenerator to facilitate such transformation.



as a motivating example, suppose we have a time series data:

$[0,1,2,3,4,5,,7,8,9]$ and we are using past 2 readings to predict the current data, then we need to transform the data as:



```

[

    [0,1]   2

    [1,2]   3

    ...

    [6,7]   8

    [7,8]   9

]

```



There are many ways to do this; However TimeseriesGenerator makes this transformation a breeze